## Análisis y Predicción de Ventas en una Tienda de Retail (Core)

### Parte III: Transformación y Análisis Avanzado de Datos con Pandas

En esta tercera parte del proyecto, continuaremos trabajando con el dataset de ventas que utilizamos en la Parte II. En esta fase, aplicaremos técnicas avanzadas de transformación y análisis de datos utilizando las nuevas habilidades adquiridas en Pandas, tales como agrupaciones complejas y el uso del método apply. Nos enfocaremos en extraer insights más profundos y preparar los datos para futuros análisis y modelos predictivos.

### Instrucciones


In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Cargamos el dataset
df = pd.read_csv('../data/retail_sales_dataset.csv')
#Visualizamos el dataset
print("Visusalizamos nuestro dataset:")
df

Visusalizamos nuestro dataset:


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100
...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,CUST996,Male,62,Clothing,1,50,50
996,997,2023-11-17,CUST997,Male,52,Beauty,3,30,90
997,998,2023-10-29,CUST998,Female,23,Beauty,4,25,100
998,999,2023-12-05,CUST999,Female,36,Electronics,3,50,150


In [25]:
'''Utiliza el método info() para obtener información general sobre el DataFrame, incluyendo el número de entradas, nombres de las columnas, tipos de datos y memoria utilizada.'''
print('Visualizamos la informacion del dataset:')
df.info()

Visualizamos la informacion del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    1000 non-null   int64 
 1   Date              1000 non-null   object
 2   Customer ID       1000 non-null   object
 3   Gender            1000 non-null   object
 4   Age               1000 non-null   int64 
 5   Product Category  1000 non-null   object
 6   Quantity          1000 non-null   int64 
 7   Price per Unit    1000 non-null   int64 
 8   Total Amount      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


In [28]:
#Verificación y ajuste de tipos de datos
#Corregimos los tipos de datos
df['Date'] = pd.to_datetime(df['Date']) # object a datetime
df['Product Category'] =df['Product Category'].astype('category') # object a category

dtypes_dict = df.dtypes.to_dict() # los demas objects a string

for column, dtype in dtypes_dict.items():
    if dtype == 'object':
        dtypes_dict[column] = 'string'
df = df.astype(dtypes_dict) 
print(df.dtypes) # imprimimos los tipos de datos arreglados

Transaction ID               int64
Date                datetime64[ns]
Customer ID         string[python]
Gender              string[python]
Age                          int64
Product Category          category
Quantity                     int64
Price per Unit               int64
Total Amount                 int64
dtype: object


In [31]:
#Verificación de duplicados
duplicados = df.duplicated().sum()
print("El dataset tiene", duplicados, "filas duplicadas")

El dataset tiene 0 filas duplicadas


In [32]:
# Verificacion de datos vacios
vacios = df.isnull().sum()
print("El dataset tiene", vacios, "filas vacias")

El dataset tiene Transaction ID      0
Date                0
Customer ID         0
Gender              0
Age                 0
Product Category    0
Quantity            0
Price per Unit      0
Total Amount        0
dtype: int64 filas vacias


In [27]:
#Generamos estadisticas descriptivas
print("Generamos estadisticas descriptivas:")
df.describe()

Generamos estadisticas descriptivas:


,Transaction ID,Age,Quantity,Price per Unit,Total Amount
count,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,500.500000,41.39200,2.514000,179.890000,456.000000
std,288.819436,13.68143,1.132734,189.681356,559.997632
min,1.000000,18.00000,1.000000,25.000000,25.000000
25%,250.750000,29.00000,1.000000,30.000000,60.000000
50%,500.500000,42.00000,3.000000,50.000000,135.000000
75%,750.250000,53.00000,4.000000,300.000000,900.000000
max,1000.000000,64.00000,4.000000,500.000000,2000.000000


In [33]:
#Revisamos los tipos de datos
df.dtypes

Transaction ID               int64
Date                datetime64[ns]
Customer ID         string[python]
Gender              string[python]
Age                          int64
Product Category          category
Quantity                     int64
Price per Unit               int64
Total Amount                 int64
dtype: object


#### Transformación de Datos
- Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.
- Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).


In [37]:
# Normalizar la columna 'Ventas'
max_ventas = df['Total Amount'].max()
min_ventas = df['Total Amount'].min()
df['Ventas_normalizado'] = df['Total Amount'].apply(lambda x: (x - min_ventas) / (max_ventas - min_ventas))

print('Ventas normalizadas:')
df['Ventas_normalizado']

Ventas normalizadas:


0      0.063291
1      0.493671
2      0.002532
3      0.240506
4      0.037975
         ...   
995    0.012658
996    0.032911
997    0.037975
998    0.063291
999    0.048101
Name: Ventas_normalizado, Length: 1000, dtype: float64

In [38]:
# Clasificar las ventas normalizadas
df['Clasificación'] = df['Ventas_normalizado'].apply(lambda x: 'alto' if x > 0.7 else 'medio' if x > 0.3 else 'bajo')
print(df)

     Transaction ID       Date Customer ID  Gender  Age Product Category  \
0                 1 2023-11-24     CUST001    Male   34           Beauty   
1                 2 2023-02-27     CUST002  Female   26         Clothing   
2                 3 2023-01-13     CUST003    Male   50      Electronics   
3                 4 2023-05-21     CUST004    Male   37         Clothing   
4                 5 2023-05-06     CUST005    Male   30           Beauty   
..              ...        ...         ...     ...  ...              ...   
995             996 2023-05-16     CUST996    Male   62         Clothing   
996             997 2023-11-17     CUST997    Male   52           Beauty   
997             998 2023-10-29     CUST998  Female   23           Beauty   
998             999 2023-12-05     CUST999  Female   36      Electronics   
999            1000 2023-04-12    CUST1000    Male   47      Electronics   

     Quantity  Price per Unit  Total Amount  Ventas_normalizado Clasificación  
0      

In [39]:
# Revisamos la clasificación en base a las ventas normalizadas
df['Clasificación'].value_counts()

Clasificación
bajo     736
medio    165
alto      99
Name: count, dtype: int64


#### Agrupación y Agregación
- Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.
- Aplicar funciones de agregación: Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.


In [51]:
# Contamos la catindad de valores unicos en la columna 'Quantity' para ver si hay valores y visualizamos cuales son
print('Valores unicos en la columna Quantity' ,df['Quantity'].nunique())
print('Valores en la columna Quantity', sorted(int(x) for x in df['Quantity'].unique()))

Valores unicos en la columna Quantity 4
Valores en la columna Quantity [1, 2, 3, 4]


In [44]:
#Generamos estadísticas descriptivas para cada combinación de 'Product Category' y 'Quantity', para identificar patrones de ventas y comportamiento del cliente según la categoría del producto y la cantidad comprada.
estadisticas = df.groupby(['Product Category','Quantity'])['Total Amount'].agg(['count','sum', 'mean', 'max', 'min', 'var', 'std'])
estadisticas

C:\Users\diego\AppData\Local\Temp\ipykernel_38496\16210610.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  estadisticas = df.groupby(['Product Category','Quantity'])['Total Amount'].agg(['count','sum', 'mean', 'max', 'min', 'var', 'std'])


count    sum        mean   max  min            var  \
Product Category Quantity                                                       
Beauty           1            74  11730  158.513514   500   25   35702.554609   
                 2            75  29930  399.066667  1000   50  166084.252252   
                 3            85  52155  613.588235  1500   75  360689.054622   
                 4            73  49700  680.821918  2000  100  548015.981735   
Clothing         1            88  16115  183.125000   500   25   38893.857759   
                 2            80  24670  308.375000  1000   50  127482.136076   
                 3            86  48315  561.802326  1500   75  303917.595759   
                 4            97  66480  685.360825  2000  100  547404.295533   
Electronics      1            91  16960  186.373626   500   25   34904.481074   
                 2            88  26450  300.568182  1000   50  144614.615987   
                 3            70  43815  625.928571  1500   75  312495.139752   
                 4            93  69680  749.247312  2000  100  584717.905563   

                                  std  
Product Category Quantity              
Beauty           1         188.951196  
                 2         407.534357  
                 3         600.573938  
                 4         740.281015  
Clothing         1         197.215257  
                 2         357.046406  
                 3         551.287217  
                 4         739.867755  
Electronics      1         186.827410  
                 2         380.282285  
                 3         559.012647  
                 4         764.668494


#### Análisis Personalizado con apply
- Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.
- Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.


In [53]:
# Definir función de desviación respecto a la media
def desviacion_venta(venta, media_grupo):
    return venta - media_grupo

# Calcular media de cada grupo y aplicar la función
df['desviacion_venta'] = df.groupby('Product Category')['Total Amount'].transform(
    lambda x: desviacion_venta(x, x.mean())
)

# Mostrar el resultado
print(df[['Product Category', 'Total Amount', 'desviacion_venta']])


    Product Category  Total Amount  desviacion_venta
0             Beauty           150       -317.475570
1           Clothing          1000        556.752137
2        Electronics            30       -428.786550
3           Clothing           500         56.752137
4             Beauty           100       -367.475570
..               ...           ...               ...
995         Clothing            50       -393.247863
996           Beauty            90       -377.475570
997           Beauty           100       -367.475570
998      Electronics           150       -308.786550
999      Electronics           120       -338.786550

[1000 rows x 3 columns]


C:\Users\diego\AppData\Local\Temp\ipykernel_38496\1652930920.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['desviacion_venta'] = df.groupby('Product Category')['Total Amount'].transform(



#### Documentación
- Comentarios claros: Documenta claramente cada paso del análisis, explicando qué se hizo y por qué se hizo.
- Código legible: Asegúrate de que el código sea legible y esté bien comentado.